In [1]:
from datasets.datamodule import DEEPscreenDataModule
from engine.system import DEEPScreenClassifier
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint
import pandas as pd

In [2]:
data = pd.read_csv("../../.data/processed/CHEMBL5567.csv")

In [4]:
trainer = Trainer(max_epochs=100)
model = DEEPScreenClassifier(fully_layer_1=256,fully_layer_2=32,drop_rate=0.5,learning_rate=0.0001,batch_size=32,experiment_result_path="../../.experiments/chembl5567")
datamodule = DEEPscreenDataModule(data=data,target_id="chembl5567",batch_size=32,experiment_result_path="../../.experiments/chembl5567",data_split_mode="non_random_split",tmp_imgs=True)
trainer.test(model,datamodule=datamodule)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '../../.experiments/chembl5567')]
INFO: Using a total of 1876 datapoints
INFO: non_random_split datasets splited train=1200,validation=300,test=376
/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.15159574151039124    │
│          test_f1          │    0.26327943801879883    │
│         test_loss         │    0.8083536624908447     │
│         test_mcc          │   -0.10020158439874649    │
│         test_pres         │    0.15159574151039124    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.8083536624908447,
  'test_acc': 0.15159574151039124,
  'test_f1': 0.26327943801879883,
  'test_mcc': -0.10020158439874649,
  'test_pres': 0.15159574151039124}]

In [12]:
trainer.test(model,datamodule=datamodule,ckpt_path="/home/sjinich/disco/TrypanoDEEPscreen/src/ml/deepscreen-epoch=105-val_loss=0.50-val_mcc=0.41-256-32-0.0001-0.5-32.ckpt")

INFO: Using a total of 1876 datapoints
INFO: non_random_split datasets splited train=1200,validation=300,test=376
Restoring states from the checkpoint path at /home/sjinich/disco/TrypanoDEEPscreen/src/ml/deepscreen-epoch=105-val_loss=0.50-val_mcc=0.41-256-32-0.0001-0.5-32.ckpt
/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.1.4, which is newer than your current Lightning version: v2.1.3
Loaded model weights from the checkpoint at /home/sjinich/disco/TrypanoDEEPscreen/src/ml/deepscreen-epoch=105-val_loss=0.50-val_mcc=0.41-256-32-0.0001-0.5-32.ckpt
/big/lab/sjinich/venv_che/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8377659320831299     │
│          test_f1          │    0.48739495873451233    │
│         test_loss         │     0.730747640132904     │
│         test_mcc          │    0.39171844720840454    │
│         test_pres         │    0.4677419364452362     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.730747640132904,
  'test_acc': 0.8377659320831299,
  'test_f1': 0.48739495873451233,
  'test_mcc': 0.39171844720840454,
  'test_pres': 0.4677419364452362}]